In [1]:
#!pip install wordcloud
#!pip install fasttext

In [1]:
import warnings
warnings.filterwarnings("ignore")

import os
import random
import numpy as np
from scipy.sparse import csr_matrix
import pandas as pd
import json

from wordcloud import WordCloud
import matplotlib.pyplot as plt
import seaborn as sns 
import matplotlib.cm as cm
from matplotlib import rcParams

import nltk 
nltk.download('stopwords')
from nltk.corpus import stopwords
from nltk.tokenize import word_tokenize
from nltk.tokenize import RegexpTokenizer
from nltk.stem.isri import ISRIStemmer
from collections import Counter 
import itertools
import re
import string

from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn import preprocessing

import fasttext

[nltk_data] Downloading package stopwords to
[nltk_data]     C:\Users\firas\AppData\Roaming\nltk_data...
[nltk_data]   Package stopwords is already up-to-date!


In [4]:
df = pd.read_csv("arabic_dataset_classifiction.csv")

In [5]:
df.head()

,text,targe
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,0
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,0
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,0
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,0
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,0


In [10]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 111728 entries, 0 to 111727
Data columns (total 2 columns):
 #   Column  Non-Null Count   Dtype 
---  ------  --------------   ----- 
 0   text    108789 non-null  object
 1   targe   111728 non-null  int64 
dtypes: int64(1), object(1)
memory usage: 1.7+ MB


## Pretraitement 

In [6]:
print(np.sum(df.isnull().any(axis=1)))

clean_df = df.dropna()
print(np.sum(clean_df.isnull().any(axis=1)))

2939
0


In [7]:
print (df.notnull().any(axis = 0))

text     True
targe    True
dtype: bool


In [8]:
def remove_hashtag(df, col = 'text'):
    for letter in r'#.][!XR':
        df[col] = df[col].astype(str).str.replace(letter,'', regex=True)

In [11]:
remove_hashtag(clean_df)
clean_df

,text,targe
0,بين أستوديوهات ورزازات وصحراء مرزوكة وآثار ولي...,0
1,قررت النجمة الأمريكية أوبرا وينفري ألا يقتصر ع...,0
2,أخبارنا المغربية الوزاني تصوير الشملالي ألهب ا...,0
3,اخبارنا المغربية قال ابراهيم الراشدي محامي سعد...,0
4,تزال صناعة الجلود في المغرب تتبع الطريقة التقل...,0
...,...,...
111723,اللاعب تأخر في العودة إلى التداريب والمدرب غاض...,4
111724,المشرف العام لحسنية أكادير قال إنه سيغادر الفر...,4
111725,نسب إليه نتائج الوداد وصحوة الرجاء وآخر صيحاته...,4
111726,ستحتضن الرباط في الفترة مابين يوليوز المقبل دو...,4


### Punctuation tretment

In [12]:
arabic_punctuations = '''`÷×؛<>_()*&^%][ـ،/:"؟.,'{}~¦+|!”…“–ـ'''
english_punctuations = string.punctuation
punctuations_list = arabic_punctuations + english_punctuations

def remove_punctuations(text):
    translator = str.maketrans('', '', punctuations_list)
    return text.translate(translator)

### Normalize

In [13]:
def normalize_arabic(text):
    text = re.sub("[إأآا]", "ا", text)
    text = re.sub("ى", "ي", text)
    text = re.sub("ة", "ه", text)
    text = re.sub("گ", "ك", text)
    return text

### Remove repetitions

In [14]:
def remove_repeating_char(text):
    return re.sub(r'(.)\1+', r'\1', text)

In [15]:
def processDocument(doc, stemmer): 

    #Replace @username with empty string
    doc = re.sub(r'@[^\s]+', ' ', doc)
    doc = re.sub(r'_', ' ', doc)
    doc = re.sub(r'\n', ' ', doc)
    doc = re.sub(r'[a-z,A-Z]', '', doc)
    doc = re.sub(r'\d', '', doc)
    #Convert www.* or https?://* to " "
    doc = re.sub('((www\.[^\s]+)|(https?://[^\s]+))',' ',doc)
    #Replace #word with word
    doc = re.sub(r'#([^\s]+)', r'\1', doc)
    # remove punctuations
    doc= remove_punctuations(doc)
    # normalize the tweet
    doc= normalize_arabic(doc)
    # remove repeated letters
    doc=remove_repeating_char(doc)
    #stemming
    doc = stemmer.stem(doc)
    
    return doc

stemmer = ISRIStemmer()
clean_df["text"] = clean_df['text'].apply(lambda x: processDocument(x, stemmer))
clean_df

,text,targe
0,بين استوديوهات ورزازات وصحراء مرزوكه واثار ولي...,0
1,قرت النجمه الامريكيه اوبرا وينفري الا يقتصر عم...,0
2,اخبارنا المغربيه الوزاني تصوير الشملالي الهب ا...,0
3,اخبارنا المغربيه قال ابراهيم الراشدي محامي سعد...,0
4,تزال صناعه الجلود في المغرب تبع الطريقه التقلي...,0
...,...,...
111723,اعب تاخر في العوده الي التداريب والمدرب غاضب و...,4
111724,مشرف العام لحسنيه اكادير قال انه سيغادر الفريق...,4
111725,نسب اليه نتائج الوداد وصحوه الرجاء واخر صيحاته...,4
111726,ستحتضن الرباط في الفتره مابين يوليوز المقبل دو...,4


In [16]:
tokenizer = RegexpTokenizer(r'\w+')
clean_df["text"] = clean_df["text"].apply(tokenizer.tokenize)
# print(clean_df['text'].values[0])
clean_df

,text,targe
0,"[بين, استوديوهات, ورزازات, وصحراء, مرزوكه, واث...",0
1,"[قرت, النجمه, الامريكيه, اوبرا, وينفري, الا, ي...",0
2,"[اخبارنا, المغربيه, الوزاني, تصوير, الشملالي, ...",0
3,"[اخبارنا, المغربيه, قال, ابراهيم, الراشدي, محا...",0
4,"[تزال, صناعه, الجلود, في, المغرب, تبع, الطريقه...",0
...,...,...
111723,"[اعب, تاخر, في, العوده, الي, التداريب, والمدرب...",4
111724,"[مشرف, العام, لحسنيه, اكادير, قال, انه, سيغادر...",4
111725,"[نسب, اليه, نتائج, الوداد, وصحوه, الرجاء, واخر...",4
111726,"[ستحتضن, الرباط, في, الفتره, مابين, يوليوز, ال...",4


In [17]:
stopwords_list = stopwords.words('arabic')
listToStr = ' '.join([str(elem) for elem in stopwords_list]) 
print(listToStr)

إذ إذا إذما إذن أف أقل أكثر ألا إلا التي الذي الذين اللاتي اللائي اللتان اللتيا اللتين اللذان اللذين اللواتي إلى إليك إليكم إليكما إليكن أم أما أما إما أن إن إنا أنا أنت أنتم أنتما أنتن إنما إنه أنى أنى آه آها أو أولاء أولئك أوه آي أي أيها إي أين أين أينما إيه بخ بس بعد بعض بك بكم بكم بكما بكن بل بلى بما بماذا بمن بنا به بها بهم بهما بهن بي بين بيد تلك تلكم تلكما ته تي تين تينك ثم ثمة حاشا حبذا حتى حيث حيثما حين خلا دون ذا ذات ذاك ذان ذانك ذلك ذلكم ذلكما ذلكن ذه ذو ذوا ذواتا ذواتي ذي ذين ذينك ريث سوف سوى شتان عدا عسى عل على عليك عليه عما عن عند غير فإذا فإن فلا فمن في فيم فيما فيه فيها قد كأن كأنما كأي كأين كذا كذلك كل كلا كلاهما كلتا كلما كليكما كليهما كم كم كما كي كيت كيف كيفما لا لاسيما لدى لست لستم لستما لستن لسن لسنا لعل لك لكم لكما لكن لكنما لكي لكيلا لم لما لن لنا له لها لهم لهما لهن لو لولا لوما لي لئن ليت ليس ليسا ليست ليستا ليسوا ما ماذا متى مذ مع مما ممن من منه منها منذ مه مهما نحن نحو نعم ها هاتان هاته هاتي هاتين هاك هاهنا هذا هذان هذه هذي هذين هكذا هل هلا هم هما هن هنا هنا

In [18]:
clean_df["text"] = clean_df["text"].apply(lambda x: [item for item in x if item not in stopwords_list])
clean_df

,text,targe
0,"[استوديوهات, ورزازات, وصحراء, مرزوكه, واثار, و...",0
1,"[قرت, النجمه, الامريكيه, اوبرا, وينفري, الا, ي...",0
2,"[اخبارنا, المغربيه, الوزاني, تصوير, الشملالي, ...",0
3,"[اخبارنا, المغربيه, قال, ابراهيم, الراشدي, محا...",0
4,"[تزال, صناعه, الجلود, المغرب, تبع, الطريقه, ال...",0
...,...,...
111723,"[اعب, تاخر, العوده, الي, التداريب, والمدرب, غا...",4
111724,"[مشرف, العام, لحسنيه, اكادير, قال, انه, سيغادر...",4
111725,"[نسب, اليه, نتائج, الوداد, وصحوه, الرجاء, واخر...",4
111726,"[ستحتضن, الرباط, الفتره, مابين, يوليوز, المقبل...",4


In [20]:
clean_df.to_csv('clean_arabic_data.csv')

In [25]:
clean_df.drop(['targe'], axis=1)

,text
0,"[استوديوهات, ورزازات, وصحراء, مرزوكه, واثار, و..."
1,"[قرت, النجمه, الامريكيه, اوبرا, وينفري, الا, ي..."
2,"[اخبارنا, المغربيه, الوزاني, تصوير, الشملالي, ..."
3,"[اخبارنا, المغربيه, قال, ابراهيم, الراشدي, محا..."
4,"[تزال, صناعه, الجلود, المغرب, تبع, الطريقه, ال..."
...,...
111723,"[اعب, تاخر, العوده, الي, التداريب, والمدرب, غا..."
111724,"[مشرف, العام, لحسنيه, اكادير, قال, انه, سيغادر..."
111725,"[نسب, اليه, نتائج, الوداد, وصحوه, الرجاء, واخر..."
111726,"[ستحتضن, الرباط, الفتره, مابين, يوليوز, المقبل..."


In [26]:
clean_df.to_csv('clear_arabic_words_.csv')

## Training

In [2]:
df = pd.read_csv('clear_arabic_words_.csv')

In [4]:
df = df[:10]

In [5]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 10 entries, 0 to 9
Data columns (total 3 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  10 non-null     int64 
 1   text        10 non-null     object
 2   targe       10 non-null     int64 
dtypes: int64(2), object(1)
memory usage: 368.0+ bytes


In [6]:
df.to_csv('test_data_arabic.csv')

In [7]:
model = fasttext.train_unsupervised('test_data_arabic.csv','skipgram')

In [8]:
model.words

["'علي',",
 "'ان',",
 "'الي',",
 "'زينه',",
 '</s>',
 "'عز',",
 "'انها',",
 "'باي',",
 "'الفيلم',",
 "'حول',",
 "'الجمهور',",
 "'الجلود',",
 "'احمد',",
 "'يتم',",
 "'الامر',",
 "'القصيره',",
 "'المغرب',",
 "'الدوره',",
 "'بالمغرب',",
 "'الفنان',",
 "'عده',",
 "'الجلد',",
 "'او',",
 "'كبير',",
 "'طنجه',",
 "'المغربيه',",
 "'الثنائي',",
 "'قائله',",
 "'القصه',",
 "'يوم',",
 "'اجل',"]

In [11]:
model.get_word_vector("يوم")

array([ 2.12028087e-03, -5.20767504e-03,  1.40864765e-02, -7.18069356e-03,
       -1.10309426e-04,  8.83226469e-03,  8.19682330e-03, -6.79837307e-03,
        3.12837190e-03,  1.32433549e-02, -1.30541045e-02,  1.46159763e-03,
        2.89452984e-03,  1.00224866e-02,  9.65427607e-03,  1.04600489e-02,
       -7.02222809e-03, -3.34338471e-03,  5.26252389e-03,  1.35787744e-02,
        1.55958941e-03, -3.86870606e-03, -6.55962480e-03,  1.67047745e-03,
        1.11163768e-03,  1.35908006e-02,  1.09178293e-03, -7.83561409e-05,
       -1.10299569e-02, -7.90364458e-04, -2.37275404e-03, -6.54473063e-03,
       -5.51900640e-03,  1.70481147e-03, -1.28492415e-02,  1.34592764e-02,
       -7.33614434e-03,  1.89285155e-03, -1.08948741e-02, -1.75955775e-03,
        1.34528696e-03,  1.76131651e-02,  9.26143862e-03, -6.69196341e-03,
        1.73279666e-03, -4.81681991e-03,  1.08280051e-02,  3.01159895e-03,
       -6.13767421e-04,  1.17161125e-02, -1.33625735e-02,  5.56377787e-03,
        6.57630328e-04,  

In [13]:
## save model 
model.save_model("models/Fasttext.bin")


In [15]:
# load model
#model = fasttext.load_model("models/Fasttext.bin")